In [ ]:
pip install xlrd
pip install openpyxl
pip install pandas

In [1]:
import pymysql
conn = pymysql.connect(host='localhost', user='db_test', password='hyesu0429@', db='dbProject')

In [2]:
curs = conn.cursor(pymysql.cursors.DictCursor)

In [1]:
import pandas as pd
import numpy as np

fileName='./movieInfo.xlsx'
df = pd.read_excel(fileName,skiprows = 4)
df = df.replace({np.nan : None})

display(df)

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,플래시,The Flash,2023.0,미국,장편,액션,개봉예정,안드레스 무시에티,None
1,엔니오: 더 마에스트로,Ennio,2021.0,이탈리아,장편,다큐멘터리,개봉예정,쥬세페 토르나토레,None
2,뉴 노멀,NEW NORMAL,2023.0,한국,장편,"스릴러,코미디",기타,정범식,None
3,명탐정코난: 하이바라 아이 이야기 ~흑철의 미스터리 트레인,Detective Conan: The Story of Ai Haibara: Blac...,2023.0,일본,장편,애니메이션,개봉예정,None,None
4,아기공룡 둘리-얼음별 대모험,The Adventure Of Ice Planet,1996.0,한국,장편,"애니메이션,어드벤처,코미디,가족",개봉,"김수정,임경원",(주)둘리나라
...,...,...,...,...,...,...,...,...,...
65526,굴레,None,2012.0,한국,단편,기타,기타,None,None
65527,미싱킴,None,2012.0,한국,단편,기타,기타,None,None
65528,피아노,None,2012.0,한국,단편,다큐멘터리,기타,None,None
65529,시간버스,None,2012.0,한국,단편,애니메이션,기타,None,None


## 테이블 생성

In [ ]:
sql_create_table_movie = "CREATE TABLE IF NOT EXISTS Movie ( id INT primary key NOT NULL AUTO_INCREMENT, name VARCHAR(45) NOT NULL, name_eng TEXT, production_year INT, type VARCHAR(10), production_status VARCHAR(10), production_company VARCHAR(100) )"
sql_create_table_director = "CREATE TABLE IF NOT EXISTS Director ( id INT primary key NOT NULL AUTO_INCREMENT, name VARCHAR(45) NOT NULL ); "
sql_create_table_movieDirector = "CREATE TABLE IF NOT EXISTS Movie_Director ( movie_id INT NOT NULL, director_id INT NOT NULL, PRIMARY KEY (movie_id, director_id), foreign key (movie_id) references Movie(id)  on update cascade on delete cascade, foreign key (director_id) references Director(id) on update cascade on delete cascade );"
sql_create_table_country = "CREATE TABLE IF NOT EXISTS Country ( id INT primary key NOT NULL AUTO_INCREMENT, name VARCHAR(20) NOT NULL );"
sql_create_table_movieCountry = "CREATE TABLE IF NOT EXISTS Movie_Country ( movie_id INT NOT NULL, country_id INT NOT NULL, PRIMARY KEY (movie_id, country_id), foreign key (movie_id) references Movie(id)  on update cascade on delete cascade, foreign key (country_id) references Country(id) on update cascade on delete cascade );"
sql_create_table_genre = "CREATE TABLE IF NOT EXISTS Genre ( movie_id INT NOT NULL ,genre VARCHAR(20) NOT NULL, PRIMARY KEY (movie_id, genre), foreign key (movie_id) references Movie(id) on update cascade on delete cascade );"

curs.execute(sql_create_table_movie)
curs.execute(sql_create_table_director)
curs.execute(sql_create_table_movieDirector)
curs.execute(sql_create_table_country)
curs.execute(sql_create_table_movieCountry)
curs.execute(sql_create_table_genre)

## 엑셀 데이터 DB에 삽입

In [4]:
sql_insert_movie = "INSERT IGNORE INTO Movie (name, name_eng, production_year, type, production_status, production_company) values(%s, %s, %s, %s, %s, %s)"

movieSet = set()

# insert into Movie 
for idx,row in df.iterrows():
    name = row['영화명']
    name_eng = row['영화명(영문)']
    production_year = row['제작연도']
    type = row['유형']
    production_status = row['제작상태']
    production_company = row['제작사']

    movie = (name, name_eng, production_year, type, production_status, production_company)
    movieSet.add(movie)
    
curs.executemany(sql_insert_movie, movieSet)
conn.commit()

print("insert success")

insert success


In [5]:
sql_insert_director = "INSERT IGNORE INTO Director (name) values(%s)"
sql_insert_country = "INSERT IGNORE INTO Country (name) values(%s)"
sql_insert_genre= "INSERT IGNORE INTO Genre (movie_id, genre) values(%s, %s)"

sql_select_id_movie= "SELECT id FROM Movie WHERE name = %s"

tuples_genre = []

movie_director_list = []
movie_country_list = []

directorSet=set()
countrySet=set()

# select id from Movie
# insert into Country, Director, Genre
for idx,row in df.iterrows():
    name = row['영화명']
    production_year = row['제작연도']
    countries = row['제작국가']
    genres = row['장르']
    directors = row['감독']
    
    curs.execute(sql_select_id_movie,name)
    movie_id = curs.fetchone()

    if directors:
        directors = directors.split(',')
        movie_director_list.append((movie_id['id'], directors))
        for director in directors:
            directorSet.add(director)
                
    if countries:
        countries = countries.split(',')
        movie_country_list.append((movie_id['id'], countries))
        for country in countries:
            countrySet.add(country)
           
    if genres:
        genres = genres.split(',')
        for genre in genres:
            tuples_genre.append((movie_id['id'], genre))


                       
curs.executemany(sql_insert_director, directorSet)
curs.executemany(sql_insert_country, countrySet)
curs.executemany(sql_insert_genre, tuples_genre)
conn.commit()

print("insert success")

insert success


In [7]:
sql_insert_movie_director= "INSERT IGNORE INTO Movie_Director (movie_id, director_id) values(%s, %s)"
sql_insert_movie_country= "INSERT IGNORE INTO Movie_Country (movie_id, country_id) values(%s, %s)"

sql_select_id_director = "SELECT id FROM Director WHERE name = %s"
sql_select_id_country = "SELECT id FROM Country WHERE name = %s"

tuples_movie_director = []
tuples_movie_country = []

# select id from Director, Country
# insert into Movie_Director, Movie_Country
for val in movie_director_list:
    movie_id = val[0]
    for director in val[1]:
        curs.execute(sql_select_id_director, director)
        director_id = curs.fetchone()
        tuples_movie_director.append((movie_id,director_id['id']))
        
for val in movie_country_list:
    movie_id = val[0]
    for country in val[1]:
        curs.execute(sql_select_id_country, country)
        country_id = curs.fetchone()
        tuples_movie_country.append((movie_id,country_id['id']))

curs.executemany(sql_insert_movie_director, tuples_movie_director)
curs.executemany(sql_insert_movie_country, tuples_movie_country)
conn.commit()

print("insert success")

insert success
